In [1]:
%pip install google-generativeai pdf2image pytesseract pillow pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import time
from pathlib import Path
from datetime import datetime
from typing import Dict, Optional, Tuple
from dotenv import load_dotenv
from jsonschema import validate, ValidationError
from supabase import create_client, Client

print("✓ All libraries imported")

✓ All libraries imported


In [4]:
PDF_DIR = r"C:\Users\samue\Documents\Work\Code\valuation_data_miner\data"
OUTPUT_DIR = "./extracted_datav3"
ERROR_LOG_DIR = "./errors"

# Load environment variables
dotenv_path = r"C:\Users\samue\Documents\Work\Code\valuation_data_miner\.env"
load_dotenv(dotenv_path)

# API Keys
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

# Model settings
GEMINI_MODEL = "models/gemini-2.5-flash" 
TESSERACT_PATH = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Create directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(ERROR_LOG_DIR, exist_ok=True)

print("✓ Configuration loaded")
print(f"  PDF Directory: {PDF_DIR}")
print(f"  Gemini Model: {GEMINI_MODEL}")

if not GEMINI_API_KEY:
    print("\n⚠ WARNING: GEMINI_API_KEY not found!")
    print("  Get key from: https://makersuite.google.com/app/apikey")

✓ Configuration loaded
  PDF Directory: C:\Users\samue\Documents\Work\Code\valuation_data_miner\data
  Gemini Model: models/gemini-2.5-flash


In [5]:
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content("Hello, respond with a JSON: {\"test\": 123}", 
                                  generation_config=genai.types.GenerationConfig(
                                      temperature=0.1,
                                      max_output_tokens=200
                                  ))

print(response.text)


c:\Users\samue\Documents\Work\Code\valuation_data_miner\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


```json
{"test": 123}
```


In [6]:
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)

models = genai.list_models()
for m in models:
    print(m.name)


models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-l

In [7]:
VALUATION_SCHEMA = {
    "type": "object",
    "required": [
        "property_id",
        "report_reference",
        "market_value_amount",
        "metadata"
    ],
    "properties": {

        # --- CORE IDENTIFIERS ---
        "property_id": {"type": "string"},
        "report_reference": {"type": "string"},

        # --- TITLE INFORMATION ---
        "title_number": {"type": "string"},
        "lr_number": {"type": "string"},
        "ir_number": {"type": "string"},

        # --- CLIENT + VALUER ---
        "client_name": {"type": "string"},
        "valuer_name": {"type": "string"},
        "inspection_date": {"type": "string"},
        "report_date": {"type": "string"},

        # --- LOCATION ---
        "location_county": {"type": "string"},
        "location_description": {"type": "string"},
        "location_coordinates": {"type": "string"},

        # --- PARCEL DETAILS ---
        "plot_area_hectares": {"type": "number"},
        "plot_area_acres": {"type": "number"},
        "land_use": {"type": "string"},

        # --- NEW LAND PHYSICAL FEATURES ---
        "plot_shape": {"type": "string"},     # rectangular, irregular, square
        "soil_type": {"type": "string"},      # black cotton, red soil, sandy, loam, murram
        "gradient": {"type": "string"},       # flat, gentle slope, moderate slope, steep
        "drainage": {"type": "string"},       # optional recommendation
        "vegetation": {"type": "string"},     # optional recommendation

        # --- TENURE + OWNERSHIP ---
        "tenure_type": {"type": "string"},
        "registered_proprietor": {"type": "string"},
        "ownership_type": {"type": "string"},
        "encumbrances": {"type": "string"},

        # --- VALUATION ---
        "market_value_amount": {"type": "number"},
        "market_value_currency": {"type": "string"},

        # --- METADATA ---
        "metadata": {
            "type": "object",
            "properties": {
                "source_file": {"type": "string"},
                "file_size_kb": {"type": "number"},
                "processing_time_seconds": {"type": "number"},
                "ocr_used": {"type": "boolean"},
                "pages_processed": {"type": "number"},
                "model_name": {"type": "string"},
                "timestamp": {"type": "string"}
            }
        }
    }
}

print("✓ Updated JSON schema defined")


✓ Updated JSON schema defined


In [8]:
def validate_extracted_data(data: Dict) -> Tuple[bool, Optional[str]]:
    """Validate extracted data against schema"""
    try:
        validate(instance=data, schema=VALUATION_SCHEMA)
        return True, None
    except ValidationError as e:
        return False, str(e)

print("✓ Validation function defined")


✓ Validation function defined


In [9]:
def init_supabase() -> Optional[Client]:
    """Initialize Supabase client"""
    try:
        if not SUPABASE_URL or not SUPABASE_KEY:
            print("⚠ Supabase credentials not configured")
            return None
        
        client = create_client(SUPABASE_URL, SUPABASE_KEY)
        print("✓ Supabase client initialized")
        return client
    except Exception as e:
        print(f"✗ Supabase init failed: {e}")
        return None

supabase_client = init_supabase()


✓ Supabase client initialized


In [10]:
SUPABASE_TABLE = "other_counties_2025"   


def sanitize_dates_for_supabase(data: dict) -> dict:
    """Convert empty string dates to None for Supabase insert"""
    date_fields = ["inspection_date", "report_date"]
    for field in date_fields:
        if field in data and (data[field] == "" or data[field] is None):
            data[field] = None
    return data


def upload_to_supabase(client: Client, data: Dict, filename: str) -> bool:
    """Upload extracted valuation data to Supabase (restricted to valid columns)."""
    if client is None:
        return False
    
    try:
        upload_data = sanitize_dates_for_supabase(data.copy())

        # Allowed columns EXACTLY as in your Supabase table
        allowed_columns = {
            "property_id",
            "report_reference",
            "title_number",
            "lr_number",
            "ir_number",
            "client_name",
            "valuer_name",
            "inspection_date",
            "report_date",
            "location_county",
            "location_description",
            "location_coordinates",
            "plot_area_hectares",
            "plot_area_acres",
            "land_use",
            "plot_shape",
            "soil_type",
            "gradient",
            "drainage",
            "vegetation",
            "tenure_type",
            "registered_proprietor",
            "ownership_type",
            "encumbrances",
            "market_value_amount",
            "market_value_currency",
            "metadata"
        }

        # Remove any field not in table
        clean_record = {k: v for k, v in upload_data.items() if k in allowed_columns}

        # Insert into your table
        client.table(SUPABASE_TABLE).insert(clean_record).execute()

        return True
    
    except Exception as e:
        raise Exception(f"Supabase upload failed: {str(e)}")


In [11]:
def process_scanned_pdf_with_ocr(pdf_path: str) -> str:
    """Convert scanned PDF to text using Tesseract OCR"""
    try:
        from pdf2image import convert_from_path
        import pytesseract
        
        # Set Tesseract path
        if os.path.exists(TESSERACT_PATH):
            pytesseract.pytesseract.tesseract_cmd = TESSERACT_PATH
        else:
            raise Exception(f"Tesseract not found at {TESSERACT_PATH}. Install from: https://github.com/UB-Mannheim/tesseract/wiki")
        
        print(f"    → Converting PDF to images...")
        
        # Convert PDF to images
        images = convert_from_path(pdf_path, dpi=300, fmt='jpeg', thread_count=2)
        
        print(f"    → Running OCR on {len(images)} pages...")
        
        # OCR each page
        all_text = []
        for i, image in enumerate(images):
            print(f"      → Page {i+1}/{len(images)}...", end=' ')
            
            text = pytesseract.image_to_string(
                image,
                lang='eng',
                config='--psm 1'
            )
            
            text = text.strip()
            
            if text:
                all_text.append(f"--- Page {i+1} ---\n{text}")
                print(f"✓ {len(text)} chars")
            else:
                print("(empty)")
        
        full_text = "\n\n".join(all_text)
        
        print(f"    → OCR complete: {len(full_text)} characters")
        
        # Save OCR output
        ocr_file = os.path.join(ERROR_LOG_DIR, f"{Path(pdf_path).stem}_ocr_text.txt")
        with open(ocr_file, 'w', encoding='utf-8') as f:
            f.write(full_text)
        
        return full_text
        
    except Exception as e:
        raise Exception(f"OCR failed: {str(e)}")

print("✓ OCR function defined")

✓ OCR function defined


In [12]:
def process_pdf_with_fallback(pdf_path: str) -> str:
    """Try multiple extraction methods"""
    
    # Try pdfplumber first
    try:
        import pdfplumber
        print(f"    → Trying pdfplumber...")
        
        all_text = []
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text:
                    all_text.append(f"--- Page {i+1} ---\n{text}")
        
        full_text = "\n\n".join(all_text)
        
        if len(full_text) > 500:
            print(f"    ✓ pdfplumber: {len(full_text)} chars")
            return full_text
        else:
            print(f"    → Only {len(full_text)} chars, trying OCR...")
    except Exception as e:
        print(f"    → pdfplumber failed, trying OCR...")
    
    # Use OCR
    return process_scanned_pdf_with_ocr(pdf_path)

print("✓ PDF extraction with fallback defined")

✓ PDF extraction with fallback defined


In [18]:
import re
import time
import json
from pathlib import Path
import google.generativeai as genai

# -----------------------------
# Helper functions
# -----------------------------
def clean_text_for_model(text: str) -> str:
    """Remove page headers, footers, and empty lines"""
    # Remove page markers like --- Page 3 ---
    text = re.sub(r'--- Page \d+ ---', '', text)
    # Remove multiple empty lines
    text = re.sub(r'\n\s*\n', '\n', text)
    return text.strip()

def find_report_reference(text: str) -> str:
    """
    Detect report reference numbers like:
    SOO/DOO/5310/1/25, SOO/5307/1/25, DOO/5307/1/25, VP/1122/24
    """
    pattern = r"\b(?:[A-Z]{2,3}/)?(?:[A-Z]{2,3}/)?\d{3,5}/\d{1,2}/\d{2,4}\b"
    matches = re.findall(pattern, text)
    if matches:
        # Choose the most complete / longest match
        return max(matches, key=len)
    return ""

def clean_gemini_json(response_text: str) -> str:
    """Extract JSON from Gemini response"""
    if '```json' in response_text:
        response_text = response_text.split('```json')[1].split('```')[0]
    elif '```' in response_text:
        response_text = response_text.split('```')[1].split('```')[0]
    response_text = response_text.strip()
    start = response_text.find('{')
    end = response_text.rfind('}')
    if start != -1 and end != -1:
        response_text = response_text[start:end+1]
    return response_text

def deep_merge(d1, d2):
    """Merge dictionaries recursively"""
    for k, v in d2.items():
        if k in d1 and isinstance(d1[k], dict) and isinstance(v, dict):
            d1[k] = deep_merge(d1[k], v)
        else:
            d1[k] = v
    return d1

# -----------------------------
# Extraction function
# -----------------------------
def extract_with_gemini(text_content: str, filename: str) -> dict:
    """
    Extract structured property valuation data from a Kenyan PDF OCR text.
    Two-pass approach: first IDs, then land/valuation.
    """

    combined_result = {}
    cleaned_text = clean_text_for_model(text_content)
    report_ref_guess = find_report_reference(cleaned_text[:3000])  # first 3k chars

    # -----------------------------
    # PASS 1: Core identifiers
    # -----------------------------
    prompt_ids = f"""
Extract the following from this Kenyan valuation report OCR text:

- report_reference (like SOO/DOO/5310/1/25)
- title_number
- lr_number
- ir_number
- client_name
- valuer_name
- inspection_date
- report_date

Use only values explicitly found in the text.
Return missing fields as empty string "".
If multiple report_reference exist, choose the longest one.
Use this as a hint for report_reference if present: "{report_ref_guess}"

Return ONLY JSON following this schema exactly:
{{
  "property_id": "",
  "report_reference": "",
  "title_number": "",
  "lr_number": "",
  "ir_number": "",
  "client_name": "",
  "valuer_name": "",
  "inspection_date": "",
  "report_date": ""
}}

TEXT:
{cleaned_text[:5000]}
"""
    try:
        model = genai.GenerativeModel("models/gemini-2.5-flash")
        response = model.generate_content(
            prompt_ids,
            generation_config=genai.types.GenerationConfig(temperature=0.1, max_output_tokens=2048)
        )
        ids_json = clean_gemini_json(response.text)
        extracted_ids = json.loads(ids_json)
        combined_result = deep_merge(combined_result, extracted_ids)
    except Exception as e:
        print(f"⚠ Gemini ID extraction failed: {e}")

    # -----------------------------
    # PASS 2: Land, valuation, metadata
    # -----------------------------
    prompt_land = f"""
Extract the following from this Kenyan valuation report OCR text:

- location_county
- location_description
- location_coordinates
- plot_area_hectares
- plot_area_acres
- land_use
- plot_shape
- soil_type
- gradient
- drainage
- vegetation
- tenure_type
- registered_proprietor
- ownership_type
- encumbrances
- market_value_amount
- market_value_currency

Use only values explicitly found in the text.
Return missing fields as "" for strings and 0 for numbers.

Return ONLY JSON following this schema exactly:
{{
  "location_county": "",
  "location_description": "",
  "location_coordinates": "",
  "plot_area_hectares": 0,
  "plot_area_acres": 0,
  "land_use": "",
  "plot_shape": "",
  "soil_type": "",
  "gradient": "",
  "drainage": "",
  "vegetation": "",
  "tenure_type": "",
  "registered_proprietor": "",
  "ownership_type": "",
  "encumbrances": "",
  "market_value_amount": 0,
  "market_value_currency": ""
}}

TEXT:
{cleaned_text}
"""
    try:
        response = model.generate_content(
            prompt_land,
            generation_config=genai.types.GenerationConfig(temperature=0.1, max_output_tokens=4096)
        )
        land_json = clean_gemini_json(response.text)
        extracted_land = json.loads(land_json)
        combined_result = deep_merge(combined_result, extracted_land)
    except Exception as e:
        print(f"⚠ Gemini Land extraction failed: {e}")

    # -----------------------------
    # Final adjustments
    # -----------------------------
    if not combined_result.get("property_id"):
        combined_result["property_id"] = Path(filename).stem
    if "report_reference" not in combined_result or not combined_result["report_reference"]:
        combined_result["report_reference"] = report_ref_guess

    # Metadata skeleton (to be populated later)
    if "metadata" not in combined_result:
        combined_result["metadata"] = {
            "source_file": filename,
            "file_size_kb": 0,
            "processing_time_seconds": 0,
            "ocr_used": True,
            "pages_processed": 0,
            "model_name": "gemini-2.5-flash",
            "timestamp": ""
        }
    # Ensure ID/date fields stay as strings
    string_fields = [
        "title_number",
        "lr_number",
        "ir_number",
        "client_name",
        "valuer_name",
        "inspection_date",
        "report_date",
    ]

    for field in string_fields:
        if field in combined_result and combined_result[field] is None:
            combined_result[field] = ""


    return combined_result


In [19]:
def process_single_pdf_complete(pdf_path: str, filename: str) -> dict:
    """Complete pipeline: OCR → Gemini → Validate → Save"""
    result = {
        'filename': filename,
        'success': False,
        'stage': None,
        'error': None,
        'data': None,
        'timing': {}
    }
    
    start_time = time.time()
    
    try:
        # Stage 1: OCR
        result['stage'] = 'ocr'
        print(f"\n  📄 Stage 1: OCR")
        stage_start = time.time()
        
        text_content = process_pdf_with_fallback(pdf_path)
        pages_processed = text_content.count('--- Page')  # approximate page count
        result['timing']['ocr'] = time.time() - stage_start
        
        if len(text_content) < 200:
            raise Exception(f"Insufficient text: {len(text_content)} chars")
        
        print(f"  ✓ {len(text_content)} chars in {result['timing']['ocr']:.1f}s")
        
        # Stage 2: Gemini
        result['stage'] = 'gemini'
        print(f"\n  🤖 Stage 2: Gemini")
        stage_start = time.time()
        
        extracted_data = extract_with_gemini(text_content, filename)
        result['timing']['gemini'] = time.time() - stage_start
        result['data'] = extracted_data
        
        print(f"  ✓ Extracted in {result['timing']['gemini']:.1f}s")
        
        # -------------------------
        # Inject real metadata
        # -------------------------
        file_size_kb = round(Path(pdf_path).stat().st_size / 1024, 2)
        processing_time = round(time.time() - start_time, 2)
        
        if "metadata" not in extracted_data:
            extracted_data["metadata"] = {}
        
        extracted_data["metadata"].update({
            "source_file": filename,
            "file_size_kb": file_size_kb,
            "processing_time_seconds": processing_time,
            "ocr_used": True,
            "pages_processed": pages_processed,
            "model_name": GEMINI_MODEL,
            "timestamp": datetime.now().isoformat()
        })
        
        # Stage 3: Validation
        result['stage'] = 'validation'
        print(f"\n  ✅ Stage 3: Validation")
        
        is_valid, validation_error = validate_extracted_data(extracted_data)
        if not is_valid:
            print(f"  ⚠ Warning: {validation_error[:100]}")
        else:
            print(f"  ✓ Valid")
        
        # Stage 4: Save
        result['stage'] = 'saving'
        print(f"\n  💾 Stage 4: Save")
        output_file = os.path.join(OUTPUT_DIR, f"{Path(filename).stem}.json")
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(extracted_data, f, indent=2, ensure_ascii=False)
        print(f"  ✓ Saved to {Path(output_file).name}")
        
        # Stage 5: Supabase
        if supabase_client and is_valid:
            result['stage'] = 'supabase'
            try:
                upload_to_supabase(supabase_client, extracted_data, filename)
                print(f"  ✓ Uploaded to Supabase")
            except Exception as e:
                print(f"  ⚠ Supabase: {str(e)[:50]}")
        
        result['success'] = True
        result['timing']['total'] = time.time() - start_time
        
    except Exception as e:
        result['error'] = str(e)
        result['timing']['total'] = time.time() - start_time
        
        error_file = os.path.join(ERROR_LOG_DIR, f"{Path(filename).stem}_error.txt")
        with open(error_file, 'w', encoding='utf-8') as f:
            f.write(f"Stage: {result['stage']}\nError: {result['error']}\n")
        
        print(f"\n  ✗ Failed: {result['error'][:100]}")
    
    return result

print("✓ Complete pipeline defined with full metadata")


✓ Complete pipeline defined with full metadata


In [20]:
def process_all_pdfs_complete():
    """Process all PDFs in the PDF_DIR, extract OCR, send to Gemini, and save JSON with full metadata."""
    
    pdf_files = list(Path(PDF_DIR).glob("*.pdf"))
    if not pdf_files:
        print(f"No PDFs found in {PDF_DIR}")
        return []

    print(f"\n{'='*70}")
    print(f"🚀 PROCESSING {len(pdf_files)} PDFs")
    print(f"{'='*70}")

    results = []
    total_start = time.time()

    for idx, pdf_path in enumerate(pdf_files, 1):

        print(f"\n{'='*70}")
        print(f"[{idx}/{len(pdf_files)}] {pdf_path.name}")
        print(f"{'='*70}")

        file_start_time = time.time()
        file_size_kb = round(pdf_path.stat().st_size / 1024, 2)

        # -------------------------
        # 1. OCR extraction
        # -------------------------
        text_content = process_pdf_with_fallback(str(pdf_path))
        pages_processed = text_content.count('--- Page')  # reliable page count from OCR text
        print(f"  ✓ OCR complete ({pages_processed} pages, {len(text_content)} chars)")

        # -------------------------
        # 2. Gemini extraction
        # -------------------------
        extracted = extract_with_gemini(text_content, pdf_path.name)

        # -------------------------
        # 3. Inject metadata block
        # -------------------------
        processing_time = round(time.time() - file_start_time, 2)
        if "metadata" not in extracted:
            extracted["metadata"] = {}

        extracted["metadata"].update({
            "source_file": pdf_path.name,
            "file_size_kb": file_size_kb,
            "processing_time_seconds": processing_time,
            "ocr_used": True,
            "pages_processed": pages_processed,
            "model_name": GEMINI_MODEL,
            "timestamp": datetime.now().isoformat()
        })

        # -------------------------
        # 4. Save JSON output
        # -------------------------
        output_path = os.path.join(OUTPUT_DIR, f"{pdf_path.stem}.json")
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(extracted, f, indent=2, ensure_ascii=False)

        print(f"✓ Saved → {output_path}")

        if supabase_client:
            try:
                upload_to_supabase(supabase_client, extracted, pdf_path.name)
                print(f"✓ Uploaded to Supabase")
            except Exception as e:
                print(f"⚠ Supabase upload failed: {str(e)[:100]}")

        results.append({
            "filename": pdf_path.name,
            "success": bool(extracted),
            "data": extracted,
            "timing": {"processing_seconds": processing_time},
        })

    # -------------------------
    # Batch summary file
    # -------------------------
    total_time = round(time.time() - total_start, 2)
    summary_file = os.path.join(OUTPUT_DIR, "summary.json")
    with open(summary_file, "w", encoding="utf-8") as f:
        json.dump({
            "timestamp": datetime.now().isoformat(),
            "model": GEMINI_MODEL,
            "total_pdfs": len(pdf_files),
            "total_time_seconds": total_time,
            "results": results,
        }, f, indent=2)

    print(f"\n🎉 Batch finished in {total_time/60:.1f} minutes")
    return results


In [16]:
def test_single_pdf():
    """Test with first PDF"""
    pdf_files = list(Path(PDF_DIR).glob("*.pdf"))
    if not pdf_files:
        print("No PDFs found")
        return
    
    test_file = pdf_files[0]
    print(f"\n🧪 TESTING: {test_file.name}\n")
    
    result = process_single_pdf_complete(str(test_file), test_file.name)
    
    if result['success']:
        print(f"\n✅ TEST PASSED!")
        print(f"\nPreview:")
        print(json.dumps(result['data'], indent=2)[:1000])
    else:
        print(f"\n❌ TEST FAILED")
    
    return result


print("\n" + "="*70)
print("="*70)
test_single_pdf()
# print("or:  results = process_all_pdfs_complete()")



🧪 TESTING: doc LR NO 7815-83 LUKENYA AREA OFF MOMBASA ROAD MACHAKOS COUNTY.pdf


  📄 Stage 1: OCR
    → Trying pdfplumber...
    → Only 0 chars, trying OCR...
    → Converting PDF to images...
    → Running OCR on 12 pages...
      → Page 1/12... ✓ 1266 chars
      → Page 2/12... ✓ 2401 chars
      → Page 3/12... ✓ 165 chars
      → Page 4/12... ✓ 182 chars
      → Page 5/12... ✓ 1357 chars
      → Page 6/12... ✓ 1800 chars
      → Page 7/12... ✓ 1664 chars
      → Page 8/12... ✓ 539 chars
      → Page 9/12... ✓ 491 chars
      → Page 10/12... ✓ 581 chars
      → Page 11/12... ✓ 1039 chars
      → Page 12/12... ✓ 607 chars
    → OCR complete: 12297 characters
  ✓ 12297 chars in 50.9s

  🤖 Stage 2: Gemini
⚠ Gemini ID extraction failed: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 2.
  ✓ Extracted in 22.2s

  ✅ Sta

{'filename': 'doc LR NO 7815-83 LUKENYA AREA OFF MOMBASA ROAD MACHAKOS COUNTY.pdf',
 'success': True,
 'stage': 'supabase',
 'error': None,
 'data': {'location_county': 'Machakos County',
  'location_description': 'The property is situated approximately 700 meters off Mombasa Road, 2.3 kilometres to the East of Greenpark Estate and 2.5 kilometres due South of Lukenya Gateway within Lukenya Atea of Machakos County.',
  'location_coordinates': '1°28\'12.2"S 37°02\'21.7"E',
  'plot_area_hectares': 2.024,
  'plot_area_acres': 5.001,
  'land_use': 'Mixed use',
  'plot_shape': 'rectangular',
  'soil_type': 'black cotton soils',
  'gradient': 'fairly level slope',
  'drainage': 'Foul drainage would be into a septic tank.',
  'vegetation': '',
  'tenure_type': 'Leasehold interest',
  'registered_proprietor': 'Kenneth Wathome Mwatu',
  'ownership_type': '',
  'encumbrances': 'Caveat registered as IR.8069/40',
  'market_value_amount': 60000000,
  'market_value_currency': 'KShs.',
  'property_id'

In [21]:
results = process_all_pdfs_complete()
results


🚀 PROCESSING 47 PDFs

[1/47] doc LR NO 7815-83 LUKENYA AREA OFF MOMBASA ROAD MACHAKOS COUNTY.pdf
    → Trying pdfplumber...
    → Only 0 chars, trying OCR...
    → Converting PDF to images...
    → Running OCR on 12 pages...
      → Page 1/12... ✓ 1266 chars
      → Page 2/12... ✓ 2401 chars
      → Page 3/12... ✓ 165 chars
      → Page 4/12... ✓ 182 chars
      → Page 5/12... ✓ 1357 chars
      → Page 6/12... ✓ 1800 chars
      → Page 7/12... ✓ 1664 chars
      → Page 8/12... ✓ 539 chars
      → Page 9/12... ✓ 491 chars
      → Page 10/12... ✓ 581 chars
      → Page 11/12... ✓ 1039 chars
      → Page 12/12... ✓ 607 chars
    → OCR complete: 12297 characters
  ✓ OCR complete (12 pages, 12297 chars)
✓ Saved → ./extracted_datav3\doc LR NO 7815-83 LUKENYA AREA OFF MOMBASA ROAD MACHAKOS COUNTY.json
✓ Uploaded to Supabase

[2/47] doc39404020250117161224 (1) TITLE NO KARAI-GIKAMBURA-6448 ONDIRI AREA KIAMBU COUNTY.pdf
    → Trying pdfplumber...
    → Only 0 chars, trying OCR...
    → Convert

[{'filename': 'doc LR NO 7815-83 LUKENYA AREA OFF MOMBASA ROAD MACHAKOS COUNTY.pdf',
  'success': True,
  'data': {'property_id': 'doc LR NO 7815-83 LUKENYA AREA OFF MOMBASA ROAD MACHAKOS COUNTY',
   'report_reference': 'SOO/DOO/5296/1/25',
   'title_number': '7815/83',
   'lr_number': '7815/83',
   'ir_number': '237281',
   'client_name': 'Mr. Kenneth Wathome',
   'valuer_name': 'Ken Wathome and Nahashon Kuria',
   'inspection_date': '14th April 2021',
   'report_date': '',
   'location_county': 'Machakos County',
   'location_description': 'The property is situated approximately 700 meters off Mombasa Road, 2.3 kilometres to the East of Greenpark Estate and 2.5 kilometres due South of Lukenya Gateway within Lukenya Atea of Machakos County.',
   'location_coordinates': '1°28\'12.2"S 37°02\'21.7"E',
   'plot_area_hectares': 2.024,
   'plot_area_acres': 5.001,
   'land_use': 'Mixed use',
   'plot_shape': 'rectangular',
   'soil_type': 'black cotton soils',
   'gradient': 'fairly level',